# Introduction to JuliAccel

## Get current version of JuliAccel.jl

### Clone the Git repository

 [ETHZ-gitlab/JuliAccel.jl](https://www.gitlab.ethz.ch/heinekas/juliaccel.jl)

### Build package

Then build the package. And install all the necessary JuliaPackages. The simpliest is from the command line running the following commands: (Make sure you put in the correct directory)

```
> julia 

julia> ]
pkg> activate /home/data/PAM/juliaccel.jl ## depends on your path
output>>>Activating project at `/home/data/ETH/HS24/PAM/juliaccel.jl`
pkg> instantiate
....
pkg> build

```
Watch out to not create a _new_ project when activating. This means you did not use the correct path.

## Load juliAccel.jl in Notebook

In [ ]:
# TODO: use the correct path for your system
import Pkg
Pkg.activate("../juliaccel.jl/")

## Basic simulation using Mad and Japd Files

The default way to use JuliAccel.jl is to define lattice in a .mad file and the used elements in a .japd file.

### Mad file

Defines the lattice of your accelerator. It uses the same file format as the Mad-x simulator.

elements 

segmenents and full lattice

beam



### Japd file

Contains a definiton of problem variables and a library of lattice element definitions including the element Hamiltonians.

__The variable definition__ definies the variables the simulation is tracking. For the typical 6-vector this is:

```
variables = (x, px, y, py, z, δ);
```

or in a simualtion that tracks only the transversal plane:

```
variables = (x, px, y, py);
```

In addition to the variables, one needs to define the truncation and expension orders.

```
orders, trunc_order = 2, exp_order = 2, order = 3;
```


__The elements__ are defined by a name (type), a list of parameters (args) and the hamiltonian (expr). For example for a drift space this is: 

```
hamiltonian, type = drift, args = (), expr = δ/β0 - sqrt((1/β0+δ)^2 - px^2 - py^2 - 1/(β0*γ0)^2);
```

### gen_maps Function

Reads in the given Mad and Japd files and returns an Array containing maps for the full lattice.

```
Ms  = JuliAccel.gen_maps(mad, prob, symplectic=true)
```


## Example using the Mad and Japd files

In [ ]:
### A basic setup


print("Importing packages... ")


using JuliAccel

using Plots
using ProgressBars

using Dates

println("Done!")


# =======================================================================

# VARIABLES

# Initial coordinates
ξi = [1.0, 0.5, 0.0, 0.0, 0.0, 0.0]


# Dragt-Finn factorise?
symp = true


# Beamline data files
# TODO make sure they are downloaded and in the same folder as the notbook!
mad  = "sample.mad"
prob = "sample.japd"



# PROBLEM DEFINITION

print("Defining problem... ")

# Compute maps
Ms  = JuliAccel.gen_maps(mad, prob, symplectic=symp)

println("Done!")



## Looking at components


In [ ]:
mad_parsed = JuliAccel.parse_madx(mad)


In [ ]:
prob_set = JuliAccel.extract_model_data(prob)

In [ ]:
Ms

## Simulation without MAD and JAPD files

In [ ]:
extract_model_data(prob)

In [ ]:

## Define beam parameters
ekin = 9.99489e8
particle = Electron()

## Calculate parameters
γ0 = get_gamma(ekin, particle.mass)
β0 = get_beta(γ)
c = 3e8

P0 = γ * β0 * particle.mass/c # *1e6#/c

# Truncation orders
n1 = 3
n2 = 6
n_max = n2 * (n1-1) + 1

## Define variables
problem_settings = eval(:(@set_problem("x px y py z δ", $n1, $n2, $n_max)))



In [ ]:
## Lattice

## ---- drift l = 1 ---- dipol l = 0.1 ---- drift l = 1 ------

l_d = 1.
l_dp = 0.1
h = 0.1
k0 = particle.charge / P0 * 1


In [ ]:
## Hamiltonians

H_drift = δ/β0 - sqrt((1/β0+δ)^2 - px^2 - py^2 - 1/(β0*γ0)^2)
H_dipol = δ/β0 - (1+ h * x) * sqrt((1/β0+δ)^2 - px^2 - py^2 - 1/(β0*γ0)^2) + (1 + 0.1 *x) * k0 *(x - (h * x^2)/(2 + 2* h *x));

In [ ]:
## generate Maps

H_drift_tpse = tps_expansion!(H_drift)
H_dipol_tpse = tps_expansion!(H_dipol);

MM = [lie_transform(-l_d * H_drift_tpse, problem_settings)]
push!(MM, lie_transform(-l_dp * H_dipol_tpse, problem_settings))
push!(MM, lie_transform(-l_d * H_drift_tpse, problem_settings))


## Propagate Particles

Take the particle state vector and loop over the lattice maps

In [ ]:
ξ = copy(ξi)

for m in MM
    ξ = m(ξ)
end

ξ

In [ ]:
k0